This notebook explores the flight data sample for the first 8 h of the first of jan 2024.

In [ ]:
from __future__ import annotations

import pandas as pd

from aia_model_contrail_avoidance.airports import list_of_uk_airports

In [2]:
parquet_file = "../../flight-data/2024_01_01_sample.parquet"
flight_dataframe = pd.read_parquet(parquet_file)
print(flight_dataframe.columns)

flight_dataframe.head()

Index(['timestamp', 'source', 'callsign', 'icao_address', 'latitude',
       'longitude', 'altitude_baro', 'altitude_gnss', 'on_ground', 'heading',
       'speed', 'vertical_rate', 'squawk', 'aircraft_type_icao', 'tail_number',
       'aircraft_typa_iata', 'aircraft_type name', 'airline_iata',
       'airline_name', 'flight_number', 'departure_airport_icao',
       'departure_airport_iata', 'arrival_airport_icao',
       'arrival_airport_iata', 'departure_scheduled_time',
       'arrival_scheduled_time', 'takeoff_time', 'landing_time',
       'arrival_utc_offset', 'departure_utc_offset', 'aircraft_type_iata',
       'aircraft_type_name', 'year', 'month', 'day', 'typecode',
       'departure_airport_name', 'arrival_airport_name', 'flight_id'],
      dtype='object')


,timestamp,source,callsign,icao_address,latitude,longitude,altitude_baro,altitude_gnss,on_ground,heading,...,departure_utc_offset,aircraft_type_iata,aircraft_type_name,year,month,day,typecode,departure_airport_name,arrival_airport_name,flight_id
0,2024-01-01 00:00:00+00:00,ADSB,TAM8085,e48ba7,29.696417,-15.103455,34000.0,NaN,False,206.138570,...,0.0,None,Boeing 777-32WER,2024,1,1,B77W,London Heathrow Airport,Guarulhos - Governador André Franco Montoro In...,35227
1,2024-01-01 00:00:00+00:00,ADSB,TOM7BX,4076b1,43.375810,-7.651850,38000.0,NaN,False,31.029434,...,0.0,None,Boeing 737-8K5,2024,1,1,B738,Lanzarote Airport,London Gatwick Airport,3650
2,2024-01-01 00:00:00+00:00,ADSB,DAL30,ab8c19,36.899864,-79.397240,33000.0,NaN,False,47.927120,...,-18000000.0,None,Boeing 767-432ER,2024,1,1,B764,Hartsfield Jackson Atlanta International Airport,London Heathrow Airport,32038
3,2024-01-01 00:00:00+00:00,ADSB,UAL929,a8a87e,48.344560,-70.525890,35000.0,33775.0,False,73.330190,...,-21600000.0,None,Boeing 767-332ER,2024,1,1,B763,Chicago O'Hare International Airport,London Heathrow Airport,30010
4,2024-01-01 00:00:00+00:00,ADSB,EZY50FH,407687,53.453140,-1.984406,4075.0,3525.0,False,336.417760,...,7200000.0,None,Airbus A320-251N,2024,1,1,A20N,Eleftherios Venizelos International Airport,Manchester Airport,3638


In [3]:
# number of flights
print(f"Number of flights: {flight_dataframe['flight_id'].nunique()}")

# destination pairs count
dest_pairs = (
    flight_dataframe.groupby(["departure_airport_name", "arrival_airport_name"])["flight_id"]
    .nunique()
    .reset_index()
)
dest_pairs = dest_pairs.rename(columns={"flight_id": "num_flights"})
dest_pairs = dest_pairs.sort_values(by="num_flights", ascending=False)
print("Top 10 destination pairs by number of flights:")
print(dest_pairs.head(10))

Number of flights: 1064
Top 10 destination pairs by number of flights:
                   departure_airport_name         arrival_airport_name  \
578        Sangster International Airport       London Gatwick Airport   
200  Grantley Adams International Airport           Manchester Airport   
217       Hewanorra International Airport       London Gatwick Airport   
117           Dubai International Airport      London Heathrow Airport   
589              Singapore Changi Airport      London Heathrow Airport   
245  John F Kennedy International Airport           Manchester Airport   
244  John F Kennedy International Airport      London Heathrow Airport   
120           Dubai International Airport            Newcastle Airport   
228   Indira Gandhi International Airport      London Heathrow Airport   
503                    Manchester Airport  Dubai International Airport   

     num_flights  
578           21  
200           17  
217           16  
117           14  
589           12  


In [9]:
# number of unique arrival-departure pairs
print(f"Number of unique arrival-departure pairs: {dest_pairs.shape[0]}")
# number of pairs with only one flight
single_flight_pairs = dest_pairs[dest_pairs["num_flights"] == 1]
print(f"Number of arrival-departure pairs with only one flight: {single_flight_pairs.shape[0]}")

# number of flights to or from heathrow
heathrow_flights = flight_dataframe[
    (flight_dataframe["departure_airport_name"] == "London Heathrow Airport")
    | (flight_dataframe["arrival_airport_name"] == "London Heathrow Airport")
]
print(f"Number of flights to or from London Heathrow: {heathrow_flights['flight_id'].nunique()}")

Number of unique arrival-departure pairs: 633
Number of arrival-departure pairs with only one flight: 471
Number of flights to or from London Heathrow: 431


In [13]:
uk_airports = list_of_uk_airports()

# number of flights to and from UK airports
uk_flights = flight_dataframe[
    (flight_dataframe["departure_airport_icao"].isin(uk_airports))
    & (flight_dataframe["arrival_airport_icao"].isin(uk_airports))
]
print(f"Number of flights to or from UK airports: {uk_flights['flight_id'].nunique()}")

# destination pairs for UK flights
uk_dest_pairs = (
    uk_flights.groupby(["departure_airport_name", "arrival_airport_name"])["flight_id"]
    .nunique()
    .reset_index()
)
uk_dest_pairs = uk_dest_pairs.rename(columns={"flight_id": "num_flights"})
uk_dest_pairs = uk_dest_pairs.sort_values(by="num_flights", ascending=False)
print("Top 10 UK destination pairs by number of flights:")
print(uk_dest_pairs.head(10))

Number of flights to or from UK airports: 55
Top 10 UK destination pairs by number of flights:
           departure_airport_name           arrival_airport_name  num_flights
13                Bristol Airport              Edinburgh Airport            2
9   Belfast International Airport        London Stansted Airport            2
30  Glasgow International Airport         London Gatwick Airport            2
43        London Stansted Airport  Belfast International Airport            2
45             Manchester Airport  Belfast International Airport            2
31  Glasgow International Airport        London Heathrow Airport            2
21              Edinburgh Airport         London Gatwick Airport            2
0           Aberdeen Dyce Airport         London Gatwick Airport            1
7   Belfast International Airport         London Gatwick Airport            1
6   Belfast International Airport  Liverpool John Lennon Airport            1


In [ ]:
from datetime import datetime

# show number of flgihts where the landing time is <8 am UTC
latest_timestamp = datetime(2024, 1, 1, 8, 0, 0)  # noqa: DTZ001
morning_landings = uk_flights[uk_flights["landing_time"] < latest_timestamp]
print(f"Number of flights landing before 8 am: {morning_landings['flight_id'].nunique()}")
list_of_uk_complete_flight_ids = list(morning_landings["flight_id"].unique())
print(f"List of flight IDs landing before 8 am: {list_of_uk_complete_flight_ids}")

Number of flights landing before 8 am: 31
List of flight IDs landing before 8 am: [np.uint32(3742), np.uint32(3409), np.uint32(3643), np.uint32(3719), np.uint32(3363), np.uint32(3747), np.uint32(3726), np.uint32(3724), np.uint32(3402), np.uint32(3456), np.uint32(3400), np.uint32(3359), np.uint32(3297), np.uint32(3579), np.uint32(3399), np.uint32(3630), np.uint32(3635), np.uint32(3738), np.uint32(3410), np.uint32(3397), np.uint32(3407), np.uint32(3404), np.uint32(3425), np.uint32(3680), np.uint32(3323), np.uint32(3371), np.uint32(3418), np.uint32(3644), np.uint32(3681), np.uint32(3411), np.uint32(3481)]


In [22]:
# show the flight ids for each pair
for _, row in uk_dest_pairs.head(10).iterrows():
    dep_airport = row["departure_airport_name"]
    arr_airport = row["arrival_airport_name"]
    flight_ids = morning_landings[
        (morning_landings["departure_airport_name"] == dep_airport)
        & (morning_landings["arrival_airport_name"] == arr_airport)
    ]["flight_id"].unique()
    print(f"Flights from {dep_airport} to {arr_airport}: {flight_ids}")

Flights from Bristol Airport to Edinburgh Airport: []
Flights from Belfast International Airport to London Stansted Airport: [3363 3726]
Flights from Glasgow International Airport to London Gatwick Airport: [3742]
Flights from London Stansted Airport to Belfast International Airport: []
Flights from Manchester Airport to Belfast International Airport: [3719 3399]
Flights from Glasgow International Airport to London Heathrow Airport: [3297]
Flights from Edinburgh Airport to London Gatwick Airport: [3409 3456]
Flights from Aberdeen Dyce Airport to London Gatwick Airport: [3738]
Flights from Belfast International Airport to London Gatwick Airport: [3418]
Flights from Belfast International Airport to Liverpool John Lennon Airport: []
